# Dog or Cat (Image classification)

## Contents

### [1. Introduction](#intro)

### [2. Data Preparation](#data)
   * **Import the required libraries**
   * **Download and unzip the dataset**
   * **Load the data**
   * **Split the data**

### [3. Model Architecture](#cnn)
   * **Set hyperparameters**
   * **Define the model**
   * **Set optimizer** 
   * **Compile model**
   * **Data Augmentation**
   * **Train model**

### [4. Model Evaluation](#eval)
   * **Training Accuracy vs Validation Accuracy**
   * **Training Loss vs Validation Loss**
   * **Model Accuracy**

### [5. Prediction](#predict)
   * **Load the test image**
   * **Predict**

<a id="intro"></a>
### 1. Introduction

#### About the dataset
The dataset is comprised of photos of dogs and cats provided as a subset of photos from a much larger dataset of 3 million manually annotated photos. The dataset was developed as a partnership between Petfinder.com and Microsoft.
The dataset was originally used as a CAPTCHA (or Completely Automated Public Turing test to tell Computers and Humans Apart), that is, a task that it is believed a human finds trivial, but cannot be solved by a machine, used on websites to distinguish between human users and bots. Specifically, the task was referred to as “ASIRRA” or Animal Species Image Recognition for Restricting Access, a type of CAPTCHA.<br>
The dataset used for in this notebook comprises of 12500 images of cats and 12500 images of dogs, totaling to 25000 images.

#### Problem statement
Given a set of images of dogs and cats. The challenge is to investigate the available data and develop an algorithm to classify whether images contain either a dog or a cat and to develop a robust test harness for estimating the performance of the model, to explore improvements to the model. Finally, load the saved model and use it to make a prediction on a single image. A final model should typically fit on all available data, such as the combination of all train and test datasets.


<a id="data"></a>
### 2. Data Preparation

#### Import the required libraries

In [ ]:
print("[INFO] Importing libraries ... ")
import os
import zipfile
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint
from shutil import move
import matplotlib.pyplot as plt

%matplotlib inline

# To ignore warnings generated during the run.
import warnings
warnings.filterwarnings("ignore")
print("[INFO] Import complete!")

#### Download and unzip the dataset

In [ ]:
local_zip = os.getcwd()+'/cats-and-dogs.zip' # Path to the local zip file.
image_dir = os.getcwd()+'/images' # Make a directory to store the extracted images.

os.listdir(os.getcwd())
# Check if the directory exists, if not then download the zip file.
if os.path.exists(local_zip) == False:
    print("[INFO] Downloading dataset ... ")
    !wget --no-check-certificate \
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
        -O "cats-and-dogs.zip"
    print("[INFO] Download complete!")

else: 
    print("[INFO] Dataset downloaded!")

# Unzip.
if len(os.listdir(image_dir)) == 0:
    print("[INFO] Unzipping dataset ... ")
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall(image_dir)
    zip_ref.close()
    print("[INFO] Unzipping complete!")

else:
    print("[INFO] Dataset unzipped!")

In [ ]:
# Verify download and extraction.
print(len(os.listdir(image_dir+'/PetImages/Cat')))
print(len(os.listdir(image_dir+'/PetImages/Dog')))
# There should be 12501 images in each of the directories.

#### Load the data

In [ ]:
# Make training and testing directories both for cat images and dog images.
try:
    os.mkdir(image_dir+'/training')
    os.mkdir(image_dir+'/testing')
    os.mkdir(image_dir+'/training/cats')
    os.mkdir(image_dir+'/training/dogs')
    os.mkdir(image_dir+'/testing/dogs')
    os.mkdir(image_dir+'/testing/cats')
except:
    pass

# Assign variables with path names to respective directories.
train_dir = image_dir+'/training'
test_dir = image_dir+'/testing'
cat_source = image_dir+'/PetImages/Cat/'
cat_train = image_dir+'/training/cats/'
cat_test = image_dir+'/testing/cats/'
dog_source = image_dir+'/PetImages/Dog/'
dog_train = image_dir+'/training/dogs/'
dog_test = image_dir+'/testing/dogs/'

#### Split the data

In [ ]:
# Function to split the dataset.
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print("[INFO] "+filename + " is of zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files)) # Randomly shuffle the test and training set.
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        move(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        move(this_file, destination)
        
split_size = 0.9 # 90% train set, 10% test set.
print("[INFO] Splitting dataset ...")
split_data(cat_source, cat_train, cat_test, split_size)
split_data(dog_source, dog_train, dog_test, split_size)
print("[INFO] Splitting complete!")

<a id="cnn"></a>
### 3. Model Architecture

#### Set hyperparameters

In [ ]:
INPUT_SHAPE = (150, 150, 3)
TARGET_SHAPE = (150,150)
BS = 20
INIT_LR = 1e-3
EPOCHS = 100
VERBOSE = 2
STEPS = 100
VAL_STEPS = 50

#### Define the model

In [ ]:
model = tf.keras.models.Sequential([
    # Layer 1
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Layer 2
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Layer 3
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Layer 4
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    # Dense Layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Output Layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#### Set optimizer 

In [ ]:
# We will use the Root Mean Square (RMS) optimizer function.
OPT = tf.keras.optimizers.RMSprop(lr = INIT_LR)

#### Compile model

In [ ]:
# Since we have to classify the image into one of the two classes, we use the binary crossentropy loss function.
model.compile(optimizer = OPT, loss='binary_crossentropy', metrics=['accuracy'])
# Get the model summary.
model.summary()

#### Data Augmentation
This helps us in exponentially increasing the dataset for training the network by applying at random a number of transformation functions to the image.<br>
It is to be noted that these functions do not alter the existing images, all these transformations are applied during the flow.

In [ ]:
# Use the image data generator class to perform image augmentation.
train_datagen = ImageDataGenerator(
      rescale = 1./255, # Rescale the pixel values so that they are <= 1.
      rotation_range = 40, # Randomly rotate the images.
      width_shift_range = 0.2, # 20% shift in widht, randomly.
      height_shift_range = 0.2, # 20% shift in height, randomly.
      shear_range = 0.2, # 20% shear, randomly.
      zoom_range = 0.2, # Randomly zoom into the images, zoom upto 20%.
      horizontal_flip = True, # Flip the image, randomly.
      fill_mode = 'nearest')

# Augmentation is only needed for the training set, not for testing set.
test_datagen = ImageDataGenerator(rescale = 1./255)

print("[INFO] Training:")
# Generate augmented data, for training images.
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size = TARGET_SHAPE,  # All images will be resized to 150x150
        class_mode = 'binary', # Since we use binary_crossentropy loss, we need binary labels
        batch_size = BS)

print("[INFO] Testing:")
# Generate augmented data, for testing images.
validation_generator = test_datagen.flow_from_directory(
        test_dir, # This is the source directory for testing images
        target_size = TARGET_SHAPE,
        class_mode = 'binary',
        batch_size = BS)

#### Train model

In [ ]:
# Create a checkpoint after every second epoch.
checkpoint = ModelCheckpoint("DogOrCat.h5", monitor = 'val_loss', verbose = 2, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)

# Train the model and record per epoch observations in history.
print("[INFO] Training model ...")
history = model.fit_generator(
      train_generator,
      steps_per_epoch = STEPS, 
      epochs = EPOCHS,
      validation_data = validation_generator,
      callbacks = [checkpoint],
      validation_steps = VAL_STEPS,
      verbose = VERBOSE)
print("[INFO] Training complete!")

<a id="eval"></a>
## 4. Model Evaluation

#### Training Accuracy vs Validation Accuracy

In [ ]:
xmin = 0
xmax = 20
ymin = 0.0
ymax = 1.0

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))


axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([ymin,ymax])

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.show()

#### Training Loss vs Validation Loss

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

axes = plt.gca()
axes.set_xlim([xmin,xmax])
axes.set_ylim([ymin,ymax])

plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.show()

#### Model Accuracy

In [ ]:
scores = model.evaluate_generator(validation_generator)
print("[INFO] Model accuracy = ", scores[1])

<a id="predict"></a>
### 5. Prediction

#### Load the test image

#### Predict